### 펜션 - 방, 날짜별 가격

숙소리스트의 상품 id를 가져와 7/1 ~ 9/30일까지의 숙소의 모든 방의 가격 정보를 저장

In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
import datetime as dt

In [6]:
# 저장한 데이터 불러오기
acc_list = pd.read_csv('../data/여기어때_숙소상세정보전체.csv').iloc[:,1:]
acc_list.head()

,category,name,score,region,ano,lat,lng,pic_url,address,pic_urls,detail
0,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
1,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-13,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
2,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
3,1,강남 캠퍼스,9.3,강남구 역삼동,4988,37.497310,127.029190,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 825-30,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
4,1,역삼 인트로호텔,9.9,강남구 역삼동,72748,37.502754,127.039923,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-14,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."


In [8]:
# 저장한 데이터 불러오기
room_list = pd.read_csv('../data/여기어때_방상세정보전체.csv')
room_detail = room_list.rename(columns={'Unnamed: 0' : "roomId"})
room_detail.head()

,roomId,accommodationId,name,checkIn,checkOut,defaultPeople,maxPeople,roomCnt
0,0,1824,랜덤객실,23.0,12,2,2,1
1,1,1824,로이,23.0,12,2,2,1
2,2,1824,샴페인골드,23.0,12,2,2,1
3,3,1824,미스트,23.0,12,2,2,1
4,4,1824,에메랄드,23.0,12,2,2,1


In [23]:
price_list = []

for index in tqdm(range(len(acc_list))):
    start = dt.datetime(2023, 7, 1)
    endDate = start.strftime('%Y-%m-%d')

    cur = acc_list.iloc[index]

    ano = cur['ano']

    while endDate != dt.datetime(2023, 10, 1).strftime('%Y-%m-%d'):
        # try:
        end = start + dt.timedelta(days=1)
        startDate = start.strftime('%Y-%m-%d')
        endDate = end.strftime('%Y-%m-%d')
        url = f"https://www.goodchoice.kr/product/detail?ano={ano}&sel_date={startDate}&sel_date2={endDate}"
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content , "html.parser")
        start = end

        html = soup.select_one("#content")
        rooms = html.select_one('.room_info')

        for room in rooms.select('.room'):
            try:
                roomCnt = room.select_one('.price').select_one('p').text.strip()
                price = room.select_one('.price').select('p')[1].text.strip()
                name = room.select_one('.title').text.strip()
                roomId = room_detail[(room_detail['accommodationId'] == ano) & (room_detail['name'] == name)].iloc[0]["roomId"]

                # room table에서 상품아이디를 primary key로 사용
                price_list.append ({
                    "roomId" : roomId,
                    "accommodationId" : ano,
                    "date" : startDate,
                    "price" : re.sub(r'[^0-9]', '', price),
                    "roomCnt" : 1,
                    "booked" : False
                })
            except:
                print('error', ano, " name : ", name)


 36%|███▌      | 95/266 [1:40:40<3:15:08, 68.47s/it]

error 49442  name :  [GLAD Morning 1+1 패키지] 스탠다드 트윈
error 49442  name :  [메디힐 마스크팩] 스탠다드 더블
error 49442  name :  [GLAD Morning 1+1 패키지] 스탠다드 더블
error 49442  name :  [GLAD in 인생네컷] 점보 트윈
error 49442  name :  [메디힐 마스크팩] 스탠다드 더블
error 49442  name :  [메디힐 마스크팩] 스탠다드 트윈
error 49442  name :  [GLAD in 인생네컷] 슈페리어 더블
error 49442  name :  [GLAD Business 패키지] 스탠다드 더블
error 49442  name :  [GLAD Morning 1+1 패키지] 스탠다드 트윈
error 49442  name :  [GLAD Morning 1+1 패키지] 스탠다드 더블
error 49442  name :  [메디힐 마스크팩] 스탠다드 더블
error 49442  name :  [메디힐 마스크팩] 스탠다드 트윈
error 49442  name :  [GLAD in 인생네컷] 슈페리어 더블
error 49442  name :  [GLAD Business 패키지] 스탠다드 더블
error 49442  name :  [GLAD in 인생네컷] 점보 트윈
error 49442  name :  [GLAD Morning 1+1 패키지] 스탠다드 트윈
error 49442  name :  [GLAD Morning 1+1 패키지] 스탠다드 더블
error 49442  name :  [메디힐 마스크팩] 스탠다드 트윈
error 49442  name :  [메디힐 마스크팩] 스탠다드 더블
error 49442  name :  [GLAD in 인생네컷] 슈페리어 더블
error 49442  name :  [GLAD Business 패키지] 스탠다드 더블
error 49442  name :  [GLAD in 인생네컷] 점보 트윈
err

 36%|███▌      | 96/266 [1:42:28<3:47:52, 80.42s/it]

error 65455  name :  로프트 더블
error 65455  name :  스튜디오 트윈
error 65455  name :  [Sunset with wine 패키지] 로프트 더블
error 65455  name :  [서울 뷰 맛집 패키지] 로프트 더블
error 65455  name :  [Sunset with wine 패키지] 스튜디오 트윈
error 65455  name :  [서울 뷰 맛집 패키지] 스튜디오 트윈
error 65455  name :  로프트 더블
error 65455  name :  스튜디오 트윈
error 65455  name :  [Sunset with wine 패키지] 로프트 더블
error 65455  name :  [서울 뷰 맛집 패키지] 로프트 더블
error 65455  name :  [Sunset with wine 패키지] 스튜디오 트윈
error 65455  name :  [서울 뷰 맛집 패키지] 스튜디오 트윈
error 65455  name :  로프트 더블
error 65455  name :  스튜디오 트윈
error 65455  name :  [Sunset with wine 패키지] 로프트 더블
error 65455  name :  [서울 뷰 맛집 패키지] 로프트 더블
error 65455  name :  [Sunset with wine 패키지] 스튜디오 트윈
error 65455  name :  [서울 뷰 맛집 패키지] 스튜디오 트윈
error 65455  name :  로프트 더블
error 65455  name :  스튜디오 트윈
error 65455  name :  [Sunset with wine 패키지] 로프트 더블
error 65455  name :  [서울 뷰 맛집 패키지] 로프트 더블
error 65455  name :  [Sunset with wine 패키지] 스튜디오 트윈
error 65455  name :  [서울 뷰 맛집 패키지] 스튜디오 트윈
error 65455  name : 

 36%|███▋      | 97/266 [1:43:25<3:26:44, 73.40s/it]

...

 99%|█████████▉| 264/266 [7:19:28<04:27, 133.86s/it]

error 74749  name :  201호
error 74749  name :  203호
error 74749  name :  103호
error 74749  name :  202호(스파)
error 74749  name :  105호
error 74749  name :  205호
error 74749  name :  101호
error 74749  name :  201호
error 74749  name :  103호
error 74749  name :  202호(스파)
error 74749  name :  105호
error 74749  name :  205호
error 74749  name :  101호
error 74749  name :  201호
error 74749  name :  203호
error 74749  name :  103호
error 74749  name :  102호(스파)
error 74749  name :  105호
error 74749  name :  205호
error 74749  name :  101호
error 74749  name :  201호
error 74749  name :  203호
error 74749  name :  103호
error 74749  name :  102호(스파)
error 74749  name :  201호
error 74749  name :  203호
error 74749  name :  103호
error 74749  name :  102호(스파)
error 74749  name :  202호(스파)
error 74749  name :  105호
error 74749  name :  205호
error 74749  name :  101호
error 74749  name :  201호
error 74749  name :  203호
error 74749  name :  103호
error 74749  name :  202호(스파)
error 74749  name :  105호
error 7474

100%|█████████▉| 265/266 [7:21:42<02:13, 133.90s/it]

error 74786  name :  C동(독채/개별수영장/마당)
error 74786  name :  C동(독채/개별수영장/마당)
error 74786  name :  S동(독채/개별수영장/마당)
error 74786  name :  A동(독채/개별수영장/마당)
error 74786  name :  C동(독채/개별수영장/마당)
error 74786  name :  D동(독채/개별수영장/마당)
error 74786  name :  C동(독채/개별수영장/마당)
error 74786  name :  S동(독채/개별수영장/마당)
error 74786  name :  S동(독채/개별수영장/마당)
error 74786  name :  A동(독채/개별수영장/마당)
error 74786  name :  B동(독채/개별수영장/마당)
error 74786  name :  D동(독채/개별수영장/마당)
error 74786  name :  A동(독채/개별수영장/마당)
error 74786  name :  B동(독채/개별수영장/마당)
error 74786  name :  C동(독채/개별수영장/마당)
error 74786  name :  D동(독채/개별수영장/마당)
error 74786  name :  S동(독채/개별수영장/마당)
error 74786  name :  A동(독채/개별수영장/마당)
error 74786  name :  B동(독채/개별수영장/마당)
error 74786  name :  C동(독채/개별수영장/마당)
error 74786  name :  D동(독채/개별수영장/마당)
error 74786  name :  S동(독채/개별수영장/마당)
error 74786  name :  A동(독채/개별수영장/마당)
error 74786  name :  B동(독채/개별수영장/마당)
error 74786  name :  C동(독채/개별수영장/마당)
error 74786  name :  D동(독채/개별수영장/마당)
error 74786  name :  S동(독채/개별수영장/마당)
e

100%|██████████| 266/266 [7:22:16<00:00, 99.76s/it] 


In [24]:
price_df = pd.DataFrame(price_list)
price_df.tail(5)

,roomId,accommodationId,date,price,roomCnt,booked
112349,1274,74633,2023-09-29,250000,1,False
112350,1275,74633,2023-09-29,650000,1,False
112351,1273,74633,2023-09-30,170000,1,False
112352,1274,74633,2023-09-30,250000,1,False
112353,1275,74633,2023-09-30,650000,1,False


In [25]:

#가격 상세정보 저장
price_df.to_csv(f"../data/여기어때_가격정보전체.csv" ,encoding="utf-8")
price_df.head(3)

,roomId,accommodationId,date,price,roomCnt,booked
0,0,1824,2023-07-01,40000,1,False
1,1,1824,2023-07-01,40000,1,False
2,2,1824,2023-07-01,40000,1,False
